In [1]:
import numpy as np
import matplotlib.pyplot as plt

from Layer import *
from Utilis import *

In [2]:
def function1(x: np.array, args=None) -> np.array: #input : 0, output : 0
    return x**2

def function2(x: np.array, args=None) -> np.array: #input : 1, output : 0
    return x[0]**2 + 2*x[1]

def function3(x: np.array, args=None) -> np.array: #input : 0, output : 1
    return np.array([3*x + 1, x**2])

def function4(x: np.array, args=None) -> np.array: #input : 1, output : 1
    return np.array([x[0]**2 + 2*x[1] - 3*x[2] + 5*x[3], 2*x[0] - 3*x[1] -x[3]**2, x[0]**3 - 3*(x[1]**2)])

def function5(x: np.array, args=None) -> np.array: #input : 1, output : 2
    return np.array([[x[0]**2 - 3*x[1], x[0]**3 - 3*(x[1]**2)], [x[0]**2, x[1]**3]])
                             
def function6(x: np.array, args=None) -> np.array: #input : 2, output : 0
    return np.array(x[0,0]**2 + x[0,1] + 2*x[1,0] - 3*x[1,1])

def function4_1(x:np.array, args=None):
    return np.array([x[0]**2, x[1]**2, x[2]**2], x[3]**2)

In [3]:
class Sigmoid:
    """Sigmoid activation function"""
    def __init__(self):
        self.type = ('activation', 'sigmoid')
        
    def __call__(self, x, args=None):
        self.x = x
        self.y = 1 / (1 + np.exp(-x))
        return self.y
    
    def backprop(self, x):
        return (1 - self.y) * self.y
    
class Softmax:
    """Softmax activation function"""
    def __init__(self):
        self.type = ('activation', 'softmax')
        self.test = True
        
    def __call__(self, x, args=None):
        self.x = x
        if self.x.ndim == 0:
            raise Exception("The arguments of the softmax function must be at least a vector. Not scalar.")
            
        elif self.x.ndim == 1:
            x = x - np.max(x)
            self.y = np.exp(x) / np.sum(np.exp(x))
            return self.y
        
        elif self.x.ndim == 2:
            x = x - np.max(x, axis=1).reshape(-1,1)
            self.y = np.exp(x) / np.sum(np.exp(x), axis=1).reshape(-1,1)
            return self.y
        
        else:
            raise Exception("Can't calculate")
    
    def backprop(self, dy):
        if self.test:
            if self.y.ndim == 1:
                soft_dy = np.diag(self.y) - np.outer(self.y, self.y)
                return soft_dy
            
            elif self.y.ndim == 2:
                batch_size = self.y.shape[0]
                num_class = self.y.shape[1]

                soft_dy = np.zeros((batch_size, num_class, num_class)) #jacobian

                for i in range(batch_size):
                    soft_dy[i] = np.diag(self.y[i]) - np.outer(self.y[i], self.y[i])
                return soft_dy
            
            else:
                raise Exception("Can't calculate")

In [4]:
"""tensors"""
x0 = np.array([0.1, 0.2, 0.3, 0.4], dtype=np.float64)
y0 = np.array([0, 1, 0, 0], dtype=np.float64)

x1 = np.array([1, 2, 3], dtype=np.float64) # 3 scalars or 1 vector
x2 = np.array([12, 13, 14], dtype=np.float64) # 3 scalars or 1 vector
x3 = np.array([x1, x2]) # 2 vectors or 1 matrix

In [5]:
sigmoid = Sigmoid()
softmax = Softmax()
relu = ReLU()

In [6]:
"""input : scalar, output : scalar"""
diff1 = differential(iterdim=0, batch=False)
temp_x = np.array(3)
print(function1(temp_x))
print(diff1(function1, temp_x))

9
6.000000000012662


In [7]:
"""input : vector, output : scalar"""
diff2 = differential(iterdim=0, batch=False)
print(function2(x0))
print(diff2(function2, x0))

0.41000000000000003
[0.2 2.  0.  0. ]


In [8]:
"""input : scalar, output : vector"""
diff3 = differential(iterdim=0, batch=False)
print(function3(temp_x))
print(diff2(function3, temp_x))

[10  9]
[3. 6.]


In [9]:
"""input : vector, output : vector"""
diff4 = differential(iterdim=0, batch=False)
print(function4(x0))
print(diff4(function4, x0))

[ 1.51  -0.56  -0.119]
[[ 0.2         2.          0.03000001]
 [ 2.         -3.         -1.2       ]
 [-3.          0.          0.        ]
 [ 5.         -0.8         0.        ]]


In [10]:
diff_sig = differential(iterdim=1)
sigmoid(x1)
print(sigmoid.backprop(x1))
print(diff_sig(sigmoid, x1))

[0.19661193 0.10499359 0.04517666]
[0.19661193 0.10499359 0.04517666]


In [11]:
diff_soft = differential(iterdim=0, batch=False)
print(softmax(x0), end="\n\n")
print(diff_soft(softmax, x0), end="\n\n")
print(softmax.backprop(x0))

[0.21383822 0.23632778 0.26118259 0.28865141]

[[ 0.16811144 -0.05053591 -0.05585082 -0.0617247 ]
 [-0.05053591  0.18047696 -0.0617247  -0.06821635]
 [-0.05585082 -0.0617247   0.19296625 -0.07539072]
 [-0.0617247  -0.06821635 -0.07539072  0.20533177]]

[[ 0.16811497 -0.05053883 -0.05585404 -0.06172209]
 [-0.05053883  0.18048056 -0.06172827 -0.06821346]
 [-0.05585404 -0.06172827  0.19296985 -0.07538754]
 [-0.06172209 -0.06821346 -0.07538754  0.20532309]]


In [12]:
print(diff_soft(Loss_Cross_Entropy, x0, y0))

[ 0.         -1.24937542  0.          0.        ]
